# 0-vanilla

Creates the most simple autoencoder.

input_dims is an integer containing the dimensions of the model input

hidden_layers is a list containing the number of nodes for each hidden layer in the encoder, respectively

the hidden layers should be reversed for the decoder

latent_dims is an integer containing the dimensions of the latent space representation

Returns: encoder, decoder, auto
encoder is the encoder model
decoder is the decoder model
auto is the full autoencoder model

The autoencoder model should be compiled using adam optimization and binary cross-entropy loss

All layers should use a relu activation except for the last layer in the decoder, which should use sigmoid

In [ ]:
#!/usr/bin/env python3
"""
Vanilla autoencoder
"""

import tensorflow.keras as keras


def autoencoder(input_dim, hidden_layers, latent_dim):
    """
    Makes a vanilla autoencoder
    """
    encoder_input = keras.layers.Input(shape=(input_dim,))
    encoder_output = encoder_input
    
    for units in hidden_layers:
        encoder_output = keras.layers.Dense(
            units, activation='relu'
        )(encoder_output)
    
    latent_space = keras.layers.Dense(
        latent_dim, activation='relu'
    )(encoder_output)
    encoder = keras.models.Model(encoder_input, latent_space)
    
    decoder_input = keras.layers.Input(shape=(latent_dim,))
    decoder_output = decoder_input
    
    for units in reversed(hidden_layers):
        decoder_output = keras.layers.Dense(
            units, activation='relu'
        )(decoder_output)
    
    decoder_output = keras.layers.Dense(
        input_dim, activation='sigmoid'
    )(decoder_output)
    decoder = keras.models.Model(decoder_input, decoder_output)
    
    auto_out = encoder(encoder_input)
    auto_out = decoder(auto_out)
    autoenc = keras.models.Model(encoder_input, auto_out)
    autoenc.compile(optimizer='adam', loss='binary_crossentropy')
    
    return encoder, decoder, autoenc


In [ ]:
# Main file

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist


autoencoder = autoencoder

# (x_train, _), (x_test, _) = mnist.load_data()
(x_train, _), (x_test, _) = fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))
np.random.seed(0)
tf.random.set_seed(0)
encoder, decoder, auto = autoencoder(784, [128, 64], 32)
auto.fit(x_train, x_train, epochs=5, batch_size=256, shuffle=True,
                validation_data=(x_test, x_test))
encoded = encoder.predict(x_test[:10])
print(np.mean(encoded))
reconstructed = decoder.predict(encoded)

for i in range(10):
    ax = plt.subplot(2, 10, i + 1)
    ax.axis('off')
    plt.imshow(x_test[i].reshape((28, 28)))
    ax = plt.subplot(2, 10, i + 11)
    ax.axis('off')
    plt.imshow(reconstructed[i].reshape((28, 28)))
plt.show()

In [ ]:
# output with cifar10 file

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

(x_train, _), (x_test, _) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = x_train.reshape((-1, 32 * 32 * 3))
x_test = x_test.reshape((-1, 32 * 32 * 3))

np.random.seed(0)
tf.random.set_seed(0)

# Build the autoencoder with the appropriate input size (32*32*3 = 3072)
encoder, decoder, auto = autoencoder(3072, [512, 256], 128)

auto.fit(x_train, x_train, epochs=10, batch_size=256, shuffle=True,
         validation_data=(x_test, x_test))

encoded = encoder.predict(x_test[:10])

print(np.mean(encoded))

reconstructed = decoder.predict(encoded)

# Plot original vs reconstructed images
for i in range(10):
    ax = plt.subplot(2, 10, i + 1)
    ax.axis('off')
    plt.imshow(x_test[i].reshape((32, 32, 3)))  # Original image

    ax = plt.subplot(2, 10, i + 11)
    ax.axis('off')
    plt.imshow(reconstructed[i].reshape((32, 32, 3)))  # Reconstructed image

plt.show()


# Task 1, Sparce autoencoder

Write a function def autoencoder(input_dims, hidden_layers, latent_dims, lambtha): that creates a sparse autoencoder:

input_dims is an integer containing the dimensions of the model input

hidden_layers is a list containing the number of nodes for each hidden layer in the encoder, respectively

the hidden layers should be reversed for the decoder

latent_dims is an integer containing the dimensions of the latent space representation

lambtha is the regularization parameter used for L1 regularization on the encoded output

Returns: encoder, decoder, auto

encoder is the encoder model
decoder is the decoder model
auto is the sparse autoencoder model
The sparse autoencoder model should be compiled using adam optimization and binary cross-entropy loss

All layers should use a relu activation except for the last layer in the decoder, which should use sigmoid

In [ ]:
#!/usr/bin/env python3
"""
Sparse autoencoder
"""

import tensorflow.keras as keras


def autoencoder(input_dim, hidden_layers, latent_dim, lambtha):
    """
    Makes a sparse autoencoder
    """
    encoder_input = keras.layers.Input(shape=(input_dim,))
    encoder_output = encoder_input
    
    for units in hidden_layers:
        encoder_output = keras.layers.Dense(
            units, activation='relu'
        )(encoder_output)
    
    latent_space = keras.layers.Dense(
        latent_dim, activation='relu',
        activity_regularizer = keras.regularizers.l1(lambtha)
    )(encoder_output)
    encoder = keras.models.Model(encoder_input, latent_space)
    
    decoder_input = keras.layers.Input(shape=(latent_dim,))
    decoder_output = decoder_input
    
    for units in reversed(hidden_layers):
        decoder_output = keras.layers.Dense(
            units, activation='relu'
        )(decoder_output)
    
    decoder_output = keras.layers.Dense(
        input_dim, activation='sigmoid'
    )(decoder_output)
    decoder = keras.models.Model(decoder_input, decoder_output)
    
    auto_out = encoder(encoder_input)
    auto_out = decoder(auto_out)
    autoenc = keras.models.Model(encoder_input, auto_out)
    autoenc.compile(optimizer='adam', loss='binary_crossentropy')
    
    return encoder, decoder, autoenc


In [ ]:
# main file

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

autoencoder = autoencoder

(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))
np.random.seed(0)
tf.random.set_seed(0)

encoder, decoder, auto = autoencoder(784, [128, 64], 32, 10e-6)
auto.fit(x_train, x_train, epochs=35,batch_size=256, shuffle=True,
                validation_data=(x_test, x_test))
encoded = encoder.predict(x_test[:10])
print(np.mean(encoded))
reconstructed = decoder.predict(encoded)

for i in range(10):
    ax = plt.subplot(2, 10, i + 1)
    ax.axis('off')
    plt.imshow(x_test[i].reshape((28, 28)))
    ax = plt.subplot(2, 10, i + 11)
    ax.axis('off')
    plt.imshow(reconstructed[i].reshape((28, 28)))